# Welcome To the TensorRT Quantization Tutorial!

This notebook is designed to show the features of the TensorRT passes integrated into MASE.

## Section 1. INT8 Quantization
Firstly, we will show you how to do a INT8 quantization of a simple model, `jsc-toy`, and compare the quantized model to the original model using the `Machop API`. The quantization process is split into the following stages, each using their own individual pass, and are explained in depth at each subsection:

1. [Fake quantization](#section-11-fake-quantization): `tensorrt_fake_quantize_transform_pass`
2. [Calibration](#sect): `tensorrt_calibrate_transform_pass`
3. [Quantized Aware Training](#quantized-aware-training): `tensorrt_fine_tune_transform_pass`
4. [Quantization](#quantization): `tensorrt_engine_interface_pass`
5. [Analysis](#analysis): `tensorrt_analysis_pass`

We start by loading in the required libraries and passes required for the notebook as well as ensuring the correct path is set for machop to be used.

In [4]:
import sys
import os
from pathlib import Path
import toml
from copy import copy, deepcopy

# Figure out the correct path
machop_path = Path(".").resolve().parent.parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

# Add directory to the PATH so that chop can be called
new_path = "../../../machop"
full_path = os.path.abspath(new_path)
os.environ['PATH'] += os.pathsep + full_path

from chop.tools.utils import to_numpy_if_tensor
from chop.tools.logger import set_logging_verbosity
from chop.tools import get_cf_args, get_dummy_input
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph
from chop.models import get_model_info, get_model, get_tokenizer
from chop.dataset import MaseDataModule, get_dataset_info
from chop.passes.graph.transforms import metadata_value_type_cast_transform_pass
from chop.passes.graph import (
    summarize_quantization_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_fake_quantize_transform_pass,
    tensorrt_fine_tune_transform_pass,
    tensorrt_engine_interface_pass,
    tensorrt_analysis_pass,
    )

set_logging_verbosity("info")

INFO     Set logging level to info
I0318 13:21:49.075905 139755614467904 logger.py:44] Set logging level to info


Next, we load in the toml file used for quantization. To view the configuration, click [here](../../machop/configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml), or read the documentation on Mase [here]().

In [5]:
# Path to your TOML file
toml_file_path = '../../../machop/configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt_quantize' section and its children
tensorrt_quantize_config = pass_args.get('passes', {}).get('tensorrt_quantize', {})
# Extract the 'passes.tensorrt_analysis' section and its children
tensorrt_analysis_config = pass_args.get('passes', {}).get('tensorrt_analysis', {})

We then create a `MaseGraph` by loading in a model and training it using the toml configuration model arguments.

In [6]:
# Load the basics in
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

# Add the data_module and other necessary information to the configs
configs = [tensorrt_quantize_config, tensorrt_analysis_config]
for config in configs:
    config['task'] = pass_args['task']
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)


input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Next, we train the `jsc-toy` model using the machop `train` action with the config from the toml file.

In [5]:
!ch train --config ../../../machop/configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml

[2024-03-18 11:44:36,191] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
INFO: Seed set to 0
I0318 11:44:37.795595 139634499045184 seed.py:54] Seed set to 0
+-------------------------+------------------------+--------------+-----------------+------------------------+
| Name                    |        Default         | Config. File | Manual Override |       Effective        |
+-------------------------+------------------------+--------------+-----------------+------------------------+
| task                    |     classification     |     cls      |                 |          cls           |
| load_name               |          None          |              |                 |          None          |
| load_type               |           mz           |              |                 |           mz           |
| batch_size              |          128           |     256      |                 |          256           |
| to_debug         

Then we load in the checkpoint. You will have to adjust this according to where it has been stored in the mase_output directory.

In [7]:
# Load in the trained checkpoint - change this accordingly
JSC_CHECKPOINT_PATH = "../../../mase_output/jsc-toy-cls_jsc/best.ckpt"

model = load_model(load_name=JSC_CHECKPOINT_PATH, load_type="pl", model=model)

# Initiate metadata
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)
mg, _ = metadata_value_type_cast_transform_pass(mg, pass_args={"fn": to_numpy_if_tensor})

# Before we begin, we will copy the original MaseGraph model to use for comparison during quantization analysis
mg_original = deepcopy_mase_graph(mg)

INFO     Loaded pytorch lightning checkpoint from ../../../mase_output/jsc-toy-cls_jsc/best.ckpt
I0318 13:22:01.808165 139755614467904 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from ../../../mase_output/jsc-toy-cls_jsc/best.ckpt


### Section 1.1 Fake Quantization

Firstly, we fake quantize the module in order to perform calibration and fine tuning before actually quantizing - this is only used if we have INT8 calibration as other precisions are not currently supported within [pytorch-quantization](https://docs.nvidia.com/deeplearning/tensorrt/pytorch-quantization-toolkit/docs/index.html#) library.

This is acheived through the `tensorrt_fake_quantize_transform_pass` which goes through the model, either by type or by name, replaces each layer appropriately to a fake quantized form if the `quantize` parameter is set in the default config (`passes.tensorrt_quantize.default.config`) or on a per name or type basis. 

Currently the quantizable layers are:
- Linear
- Conv1d, Conv2d, ConvNd 
- ConvTranspose1d, ConvTranspose2d, ConvTransposeNd 
- MaxPool1d, MaxPool2d, MaxPool3d
- AvgPool1d, AvgPool2d, AvgPool3d
- Clip (Tensor)
- LSTM, LSTMCell

To create a custom quantized module, click [here](https://docs.nvidia.com/deeplearning/tensorrt/pytorch-quantization-toolkit/docs/index.html#document-tutorials/creating_custom_quantized_modules).


In [8]:
mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_quantize_config)
summarize_quantization_analysis_pass(mg_original, mg)

INFO     Applying fake quantization to PyTorch model...
I0318 13:22:13.968679 139755614467904 utils.py:167] Applying fake quantization to PyTorch model...
INFO     Fake quantization applied to PyTorch model.
I0318 13:22:14.145424 139755614467904 utils.py:192] Fake quantization applied to PyTorch model.
INFO     Quantized graph histogram:
I0318 13:22:14.156768 139755614467904 summary.py:84] Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       4 |         0 |           4 |
| Linear          | linear       |       3 |         3 |           0 |
| ReLU            | relu         |       4 |         0 |           4 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |
I0318 13:22:14.158703 139755614467904 summary.py:85] 
| Original type   |

As you can see we have succesfully quantized all linear layers inside `jsc-toy`. See [Section 4](#section-4-layer-wise-mixed-precision) for how to apply quantization layerwise.

### Section 1.2 Calibration

Next, we perform calibration using the `tensorrt_calibrate_transform_pass`. Calibration is achieved by passing data samples to the quantizer and deciding the best amax for activations. 

Calibrators can be added as a search space parameter to examine the best performing calibrator. The calibrators have been included in the toml as follows.
For example: `calibrators = ["percentile", "mse", "entropy"]`

Note: 
- To use `percentile` calibration, a list of percentiles must be given
- To use `max` calibration, the `histogram` weight and input calibrators must be removed and replaced with `max`. This will use global maximum absolute value to calibrate the model.
- If `post_calibration_analysis` is set true the `tensorrt_analysis_pass` will be run for each calibrator tested to evaluate the most suitable calibrator for the model.

In [9]:
mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_quantize_config)

INFO     Starting calibration of the model in PyTorch...
I0318 13:22:19.642365 139755614467904 calibrate.py:91] Starting calibration of the model in PyTorch...
INFO     Disabling Quantization and Enabling Calibration
I0318 13:22:19.651999 139755614467904 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 13:22:19.653707 139755614467904 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 13:22:19.655657 139755614467904 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 13:22:19.657295 139755614467904 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 13:22:19.658708 139755614467904 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and

From the results, the 99% `percentile` clips too many values during the amax calibration, comprimising the loss. However 99.99% demonstrates higher validation accuracy alongside `mse` and `entropy` for `jsc-toy`. For such a small model, the methods are not highly distinguished, however for larger models this calibration process will be important for ensuring the quantized model still performs well. 

### Section 1.3 Quantized Aware Training (QAT)

The `tensorrt_fine_tune_transform_pass` is used to fine tune the quantized model. 

For QAT it is typical to employ 10% of the original training epochs, starting at 1% of the initial training learning rate, and a cosine annealing learning rate schedule that follows the decreasing half of a cosine period, down to 1% of the initial fine tuning learning rate (0.01% of the initial training learning rate). However this default can be overidden by setting the `epochs`, `initial_learning_rate` and `final_learning_rate` in `passes.tensorrt_quantize.fine_tune`.

The fine tuned checkpoints are stored in the ckpts/fine_tuning folder:

```
mase_output
└── tensorrt
    └── quantization
        ├── cache
        ├── ckpts
        │   └── fine_tuning
        ├── json
        ├── onnx
        └── trt

```

In [10]:
mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_quantize_config)

INFO     Starting Fine Tuning...
I0318 13:23:20.128491 139755614467904 fine_tune.py:62] Starting Fine Tuning...
INFO     Fine tuninig for 2 epochs
I0318 13:23:20.170033 139755614467904 fine_tune.py:102] Fine tuninig for 2 epochs
I0318 13:23:20.252301 139755614467904 rank_zero.py:64] GPU available: True (cuda), used: True
I0318 13:23:20.266848 139755614467904 rank_zero.py:64] TPU available: False, using: 0 TPU cores
I0318 13:23:20.267485 139755614467904 rank_zero.py:64] IPU available: False, using: 0 IPUs
I0318 13:23:20.268031 139755614467904 rank_zero.py:64] HPU available: False, using: 0 HPUs
I0318 13:23:20.274132 139755614467904 rank_zero.py:64] You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float

/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Epoch 1: 100%|██████████| 3084/3084 [00:57<00:00, 53.90it/s, v_num=4, train_acc_step=0.765, val_acc_epoch=0.733, val_loss_epoch=0.750]

I0318 13:25:17.216056 139755614467904 rank_zero.py:64] `Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 3084/3084 [00:57<00:00, 53.90it/s, v_num=4, train_acc_step=0.765, val_acc_epoch=0.733, val_loss_epoch=0.750]

INFO     Fine Tuning Complete
I0318 13:25:17.223005 139755614467904 fine_tune.py:121] Fine Tuning Complete


### Section 1.4 TensorRT Quantization

After QAT, we are now ready to convert the model to a tensorRT engine so that it can be run with the superior inference speeds. To do so, we use the `tensorrt_engine_interface_pass` which converts the `MaseGraph`'s model from a Pytorch one to an ONNX format as an intermediate stage of the conversion.

During the conversion process, the `.onnx` and `.trt` files are stored to their respective folders shown in [Section 1.3](#section-13-quantized-aware-training-qat).

This interface pass returns a dictionary containing the `onnx_path` and `trt_engine_path`.

In [12]:
mg, meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_quantize_config)

INFO     Converting PyTorch model to ONNX...
I0318 13:27:06.409828 139755614467904 quantize.py:129] Converting PyTorch model to ONNX...
INFO     ONNX Conversion Complete. Stored ONNX model to /root/mase/mase_output/tensorrt/quantization/onnx/2024_03_18/version_10/model.onnx
I0318 13:27:31.306156 139755614467904 quantize.py:152] ONNX Conversion Complete. Stored ONNX model to /root/mase/mase_output/tensorrt/quantization/onnx/2024_03_18/version_10/model.onnx
INFO     Converting PyTorch model to TensorRT...
I0318 13:27:37.806547 139755614467904 quantize.py:55] Converting PyTorch model to TensorRT...
INFO     TensorRT Conversion Complete. Stored trt model to /root/mase/mase_output/tensorrt/quantization/trt/2024_03_18/version_6/model.trt
I0318 13:27:46.186066 139755614467904 quantize.py:124] TensorRT Conversion Complete. Stored trt model to /root/mase/mase_output/tensorrt/quantization/trt/2024_03_18/version_6/model.trt
INFO     TensorRT Model Summary Exported to /root/mase/mase_output/tensor

### Section 1.5 Performance Analysis

To showcase the improved inference speeds and to evaluate accuracy and other performance metrics, the `tensorrt_analysis_pass` can be used.

The tensorRT engine path obtained the previous interface pass is now inputted into the the analysis pass. The same pass can take a MaseGraph as an input, as well as an ONNX graph. For this comparison, we will first run the anaylsis pass on the original unquantized model and then on the INT8 quantized model.

In [13]:
_, _ = tensorrt_analysis_pass(mg_original, pass_args=tensorrt_analysis_config)
_, _ = tensorrt_analysis_pass(meta['trt_engine_path'], pass_args=tensorrt_analysis_config)

INFO     Starting transformation analysis
I0318 13:28:06.032049 139755614467904 analysis.py:214] Starting transformation analysis
INFO     
Results jsc-toy:
+------------------------------+--------------+
|            Metric            |    Value     |
+------------------------------+--------------+
|    Average Test Accuracy     |   0.73168    |
|      Average Precision       |   0.74472    |
|        Average Recall        |   0.73037    |
|       Average F1 Score       |   0.73369    |
|         Average Loss         |   0.76315    |
|       Average Latency        |  1.0634 ms   |
|   Average GPU Power Usage    |   59.251 W   |
| Inference Energy Consumption | 0.017502 mWh |
+------------------------------+--------------+
I0318 13:28:11.978660 139755614467904 analysis.py:330] 
Results jsc-toy:
+------------------------------+--------------+
|            Metric            |    Value     |
+------------------------------+--------------+
|    Average Test Accuracy     |   0.73168    |
| 

As shown above, the latency has decreased around 4x with the `jsc-toy` model without compromising accuracy due to the well calibrated amax and quantization-aware fine tuning. The inference energy consumption has thus also dropped tremendously and this is an excellent demonstration for the need to quantize in industry especially for LLMs in order to reduce energy usage. 

## Section 2. FP16 Quantization

We will now load in a new toml configuration that uses FP16 instead of INT8, whilst keeping the other settings the exact same for a fair comparison. This time however, we will use chop from the terminal which runs all the passes showcased in [Section 1](#section-1---int8-quantization).

Since float quantization does not require calibration, nor is it supported by `pytorch-quantization`, the model will not undergo fake quantization, unfortunately, for the time being this means QAT is unavailable.

In [5]:
!ch transform --config ../../../machop/configs/tensorrt/jsc_toy_FP16_quantization_by_type.toml --load {JSC_CHECKPOINT_PATH} --load-type pl

[2024-03-18 13:15:17,844] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
INFO: Seed set to 0
I0318 13:15:19.415035 140626882156352 seed.py:54] Seed set to 0
+-------------------------+------------------------+--------------+--------------------------+--------------------------+
| Name                    |        Default         | Config. File |     Manual Override      |        Effective         |
+-------------------------+------------------------+--------------+--------------------------+--------------------------+
| task                    |     classification     |     cls      |                          |           cls            |
| load_name               |          None          |              | /root/mase/mase_output/j | /root/mase/mase_output/j |
|                         |                        |              | sc-toy-cls_jsc/best.ckpt | sc-toy-cls_jsc/best.ckpt |
| load_type               |           mz           |            

In [16]:
!ch transform --config ../../../machop/configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml --load {JSC_CHECKPOINT_PATH} --load-type pl

2136.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
[2024-03-18 12:26:37,519] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
INFO: Seed set to 0
I0318 12:26:39.119276 139887962613568 seed.py:54] Seed set to 0
+-------------------------+------------------------+--------------+--------------------------+--------------------------+
| Name                    |        Default         | Config. File |     Manual Override      |        Effective         |
+-------------------------+------------------------+--------------+--------------------------+--------------------------+
| task                    |     classification     |     cls      |                          |           cls            |
| load_name               |          None          |              | /root/mase/mase_output/j | /root/mase/mase_output/j |
|                         |                        |              | sc-toy-cls_jsc/best.ckp

As you can see, `FP16` acheives a slighty higher test accuracy and a slightly lower latency (~10%) from that of INT8 quantization as expected. Now lets try a more complicated model - `vgg7`.

## Section 3. Type-wise Mixed Precision on Larger Model
We will now quantize `vgg7` which includes both convolutional and linear layers, however for this demonstration we want to quantize all layer types except the linear layers.

In this case, we set:

- The `by` parameter to `type`
- The `quantize` parameter to true for `passes.tensorrt_quantize.conv2d.config` and `precision` parameter to 'INT8'.
- The `input` and `weight` quantize axis for the conv2d layers.
- The default `passes.tensorrt_quantize.default.config` to false. 

During the TensorRT quantization, the model's conv2d layers will be converted to an INT8 fake quantized form, whilst the linear layers are kept to their default 'FP16'. Calibration of the conv2d layers will be undergone and fine tuning.  

You may either download a pretrained model [here](https://imperiallondon-my.sharepoint.com/:f:/g/personal/zz7522_ic_ac_uk/Emh3VT7Q_qRFmnp8kDrcgDoBwGUuzLwwKNtX8ZAt368jJQ?e=gsKONa), otherwise train it yourself as shown below. 

In [ ]:
!ch train --config ../../../machop/configs/tensorrt/vgg7_FP16_quantization_by_type.toml

We will now load the checkpoint in and quantize the model and compare it to the unquantized version as we did in [Section 1.5](#section-15-performance-analysis)

In [1]:
VGG_CHECKPOINT_PATH = "../../../mase_output/vgg7-pre-trained/test-accu-0.9332.ckpt"

In [2]:
!ch transform --config ../../../machop/configs/tensorrt/vgg7_INT8_quantization_by_type.toml --load {VGG_CHECKPOINT_PATH} --load-type pl

/bin/bash: line 1: ch: command not found


# Section 4. Layer-wise Mixed Precision

So far we have strictly quantized either in INT8 or FP16. Now, we will show how to conduct layerwise mixed precision using the same `vgg7` model. In this case we will show how for instance, layer 0 and 1 can be set to FP16, while layer 2 and 3 can be INT8 quantized. 

For this, we set:
- The `by` parameter to `name`
- The `precision` to 'FP16' for `passes.tensorrt_quantize.feature_layers_0.config and passes.tensorrt_quantize.feature_layers_1.config`
- The `precision` to 'INT8' for `passes.tensorrt_quantize.feature_layers_0.config and passes.tensorrt_quantize.feature_layers_1.config`



In [1]:
import sys
import os
from pathlib import Path
import toml
from copy import copy, deepcopy

# Figure out the correct path
machop_path = Path(".").resolve().parent.parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

# Add directory to the PATH so that chop can be called
new_path = "../../../machop"
full_path = os.path.abspath(new_path)
os.environ['PATH'] += os.pathsep + full_path

from chop.tools.utils import to_numpy_if_tensor
from chop.tools.logger import set_logging_verbosity
from chop.tools import get_cf_args, get_dummy_input
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph
from chop.models import get_model_info, get_model, get_tokenizer
from chop.dataset import MaseDataModule, get_dataset_info
from chop.passes.graph.transforms import metadata_value_type_cast_transform_pass
from chop.passes.graph import (
    summarize_quantization_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_fake_quantize_transform_pass,
    tensorrt_fine_tune_transform_pass,
    tensorrt_engine_interface_pass,
    tensorrt_analysis_pass,
    )

set_logging_verbosity("info")

/opt/conda/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-18 11:38:55,375] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO     Set logging level to info
I0318 11:38:56.826868 139779632510784 logger.py:44] Set logging level to info


In [2]:
# Path to your TOML file
toml_file_path = '../../../machop/configs/tensorrt/vgg7_layerwise_mixed_precision.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt_quantize' section and its children
tensorrt_quantize_config = pass_args.get('passes', {}).get('tensorrt_quantize', {})
# Extract the 'passes.tensorrt_analysis' section and its children
tensorrt_analysis_config = pass_args.get('passes', {}).get('tensorrt_analysis', {})

# Load the basics in
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

# Add the data_module and other necessary information to the configs
configs = [tensorrt_quantize_config, tensorrt_analysis_config]
for config in configs:
    config['task'] = pass_args['task']
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)


input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Load in the trained checkpoint - change this accordingly
VGG_CHECKPOINT_PATH = "../../../mase_output/vgg7-pre-trained/test-accu-0.9332.ckpt"

model = load_model(load_name=VGG_CHECKPOINT_PATH, load_type="pl", model=model)

# Initiate metadata
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)
mg, _ = init_metadata_analysis_pass(mg, None)

mg_original = deepcopy_mase_graph(mg)

mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)
mg, _ = metadata_value_type_cast_transform_pass(mg, pass_args={"fn": to_numpy_if_tensor})

INFO     Loaded pytorch lightning checkpoint from ../../../mase_output/vgg7-pre-trained/test-accu-0.9332.ckpt
I0318 11:23:31.613022 140342552557376 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from ../../../mase_output/vgg7-pre-trained/test-accu-0.9332.ckpt


In [10]:
_, _ = tensorrt_analysis_pass(mg, pass_args=tensorrt_analysis_config)

INFO     Starting transformation analysis
I0318 09:55:37.107798 140195009255232 analysis.py:214] Starting transformation analysis
INFO     
Results vgg7:
+------------------------------+-------------+
|            Metric            |    Value    |
+------------------------------+-------------+
|    Average Test Accuracy     |   0.91967   |
|      Average Precision       |   0.92315   |
|        Average Recall        |   0.92362   |
|       Average F1 Score       |   0.92326   |
|         Average Loss         |   0.23674   |
|       Average Latency        |  28.123 ms  |
|   Average GPU Power Usage    |  100.17 W   |
| Inference Energy Consumption | 0.78256 mWh |
+------------------------------+-------------+
I0318 09:55:40.969246 140195009255232 analysis.py:330] 
Results vgg7:
+------------------------------+-------------+
|            Metric            |    Value    |
+------------------------------+-------------+
|    Average Test Accuracy     |   0.91967   |
|      Average Precision

In [5]:
mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_quantize_config)
# summarize_quantization_analysis_pass(mg_original, mg)
mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_quantize_config)
mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_quantize_config)
mg, meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_quantize_config)

INFO     Applying fake quantization to PyTorch model...
I0318 11:28:17.732462 140342552557376 utils.py:199] Applying fake quantization to PyTorch model...


INFO     Fake quantization applied to PyTorch model.
I0318 11:28:18.571809 140342552557376 utils.py:224] Fake quantization applied to PyTorch model.
INFO     Starting calibration of the model in PyTorch...
I0318 11:28:18.574708 140342552557376 calibrate.py:91] Starting calibration of the model in PyTorch...
INFO     Disabling Quantization and Enabling Calibration
I0318 11:28:18.593663 140342552557376 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 11:28:18.595410 140342552557376 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 11:28:18.596705 140342552557376 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 11:28:18.597985 140342552557376 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Ca

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


I0318 11:31:02.771356 140342552557376 cuda.py:61] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
I0318 11:31:02.798513 140342552557376 model_summary.py:94] 
  | Name      | Type               | Params
-------------------------------------------------
0 | model     | GraphModule        | 14.0 M
1 | loss_fn   | CrossEntropyLoss   | 0     
2 | acc_train | MulticlassAccuracy | 0     
3 | loss_val  | MeanMetric         | 0     
4 | loss_test | MeanMetric         | 0     
-------------------------------------------------
14.0 M    Trainable params
0         Non-trainable params
14.0 M    Total params
56.118    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

...

I0318 11:33:01.304579 140342552557376 rank_zero.py:64] `Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 196/196 [00:58<00:00,  3.35it/s, v_num=1, train_acc_step=0.875, val_acc_epoch=0.896, val_loss_epoch=0.219]

INFO     Fine Tuning Complete
I0318 11:33:01.409875 140342552557376 fine_tune.py:121] Fine Tuning Complete
INFO     Converting PyTorch model to ONNX...
I0318 11:33:01.418860 140342552557376 quantize.py:129] Converting PyTorch model to ONNX...


/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_quantization/tensor_quant.py:363: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if min_amax < 0:
/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_quantization/tensor_quant.py:366: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  max_bound = torch.tensor((2.0**(num_bits - 1 + int(unsigned))) - 1.0, device=amax.device)
/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_quantization/tensor_quant.py:376: TracerWarning: Converting a tensor to a Pytho

KeyboardInterrupt: 

In [ ]:
# !ch transform --config ../../../machop/configs/tensorrt/vgg7_INT8_quantization_by_type.toml --load {VGG_CHECKPOINT_PATH} --load-type pl

## Section 5. Language Models

In [2]:
# Path to your TOML file
toml_file_path = '../../../machop/configs/tensorrt/opt-125M_layerwise_mixed_precision_by_name.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt_quantize' section and its children
tensorrt_quantize_config = pass_args.get('passes', {}).get('tensorrt_quantize', {})
# Extract the 'passes.tensorrt_analysis' section and its children
tensorrt_analysis_config = pass_args.get('passes', {}).get('tensorrt_analysis', {})

# Load the basics in
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

opt_tokenizer = get_tokenizer("facebook/opt-125m")

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=os.cpu_count(),
    max_token_len=128,
    tokenizer=opt_tokenizer,
    load_from_cache_file=True,
)
data_module.prepare_data()
data_module.setup()

# Add the data_module and other necessary information to the configs
configs = [tensorrt_quantize_config, tensorrt_analysis_config]
for config in configs:
    config['task'] = pass_args['task']
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

In [3]:
model_info = get_model_info(model_name)
model = get_model(
    "facebook/opt-125m:patched",
    task="lm",
    dataset_info=get_dataset_info("wikitext2"),
    pretrained=True,
)

# Load in the trained checkpoint - change this accordingly
# OPT125M_CHECKPOINT_PATH = "../../../mase_output/jsc-toy_classification_jsc_2024-03-17/software/training_ckpts/best.ckpt"
# model = load_model(load_name=OPT125M_CHECKPOINT_PATH, load_type="pl", model=model)

model_info = get_model_info("facebook/opt-125m:patched")
cf_args = get_cf_args(model_info=model_info, task="lm", model=model)

mg = MaseGraph(model=model, cf_args=cf_args)

# dummy_in = get_dummy_input(model_info, data_module=data_module, task="lm")
# if len(mg.model.additional_inputs) > 0:
#     dummy_in = dummy_in | mg.model.additional_inputs

# Initiate metadata
mg, _ = init_metadata_analysis_pass(mg, pass_args=None)

# # Before we begin, we will copy the original MaseGraph model to use for comparison during quantization analysis
# mg_original = deepcopy_mase_graph(mg)

In [ ]:
# _, _ = tensorrt_analysis_pass(mg, pass_args=tensorrt_analysis_config)

In [4]:
# mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_quantize_config)
# summarize_quantization_analysis_pass(mg_original, mg)

# mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_quantize_config)

# mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_quantize_config)

mg, meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_quantize_config)

_, _ = tensorrt_analysis_pass(mg_original, pass_args=tensorrt_analysis_config)
_, _ = tensorrt_analysis_pass(meta['trt_engine_path'], pass_args=tensorrt_analysis_config)

INFO     Converting PyTorch model to ONNX...
I0318 08:54:47.010779 139760749061952 quantize.py:129] Converting PyTorch model to ONNX...
ERROR:tornado.general:SEND Error: Host unreachable
Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 577, in _pydevd_bundle.pydevd_cython.PyDBFrame._handle_exception
  File "_pydevd_bundle/pydevd_cython.pyx", line 312, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/opt/conda/envs/mase/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2070, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/mase/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2106, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt
